In [ ]:
# !sed -i 's/transformers.generation_utils/transformers.generation/' /usr/local/lib/python3.12/dist-packages/openprompt/pipeline_base.py

In [1]:
# Set device
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
# ==============================
# Imports
# ==============================
import pandas as pd
import torch
import random
import numpy as np
from collections import Counter
from transformers import XLMRobertaTokenizer, XLMRobertaForMaskedLM
from openprompt.prompts import ManualTemplate, ManualVerbalizer
from openprompt.prompts import SoftVerbalizer
from openprompt.prompts import AutomaticVerbalizer
from openprompt.data_utils import InputExample
from openprompt.plms import load_plm
from openprompt import PromptForClassification, PromptDataLoader
from torch.optim import AdamW
from sklearn.metrics import classification_report

C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\transformers\generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


In [3]:


# ==============================
# Set random seeds for reproducibility
# ==============================
# 💡 Added this block to ensure consistent results across runs
seed = 20
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [4]:
# ==============================
# Step 1: Training data (3-shot / few-shot)
# ==============================
train_dataset = [
    #Politics
    InputExample(guid=0, text_a="بلوچستان ۾ ايراني سرزمين تان ٻيهر حملو، پاڪ فوج جا 4 جوان شهيد", label=0),
    InputExample(guid=1, text_a="سپريم ڪورٽ ۾ قرآن پاڪ جي حافظ کي 20 واڌو نمبر ڏيڻ بابت پاڻمرادي نوٽيس ۾ جسٽس قاضي فائز عيسيٰ جي سربراهي ۾ 3 رڪني خاص بئنچ فيصلو جاري ڪيو", label=0),
    InputExample(guid=2, text_a="واضح رهي ته ڪيرول ئي نه پر ڪيترين ئي عورتن اڳوڻي آمريڪي صدر تي جنسي ڏاڍائي ۾ ملوث هجڻ جا الزام لڳايا آهن.", label=0),
    InputExample(guid=3, text_a="پيادل قافلي جي ڀرپور آجيان ڪري مٿن گلن جي ورکا ڪئي وئي.", label=0),

    #Sports
    InputExample(guid=4, text_a="نيپال جي ڪرڪيٽ ميچ ڊي ايل ايس ميٿڊ تحت ون ڊي ۾ 9 وڪيٽن تي يو اي اي جي ٽيم کي شڪست ڏئي ڇڏي آهي", label=1),
    InputExample(guid=5, text_a="هوڏانهن ٻئي سيمي فائنل ۾ وري آريانا سيبالينڪا يونان جي ماريا سڪاري کي شڪست ڏئي فائنل ۾ جڳهه ٺاهي آهي، هاڻ فائنل ۾ آريانا سيبالينڪا ۽ ايلانا ريباڪنا آمهون سامهون ٿينديون.", label=1),
    InputExample(guid=6, text_a="جڏهن تھ سريلنڪا ويمن جي ٽيم 19 اوور ۾ رڳو 3 وڪيٽن جي نقصان تي هدف پورو ڪري ڪاميابي ماڻي ورتي، جن ۾ چماري اتاپٽو 33، هرشيٿا سماراوڪرما 29، ڪويشا ڊلهاري 20 رنسن سان نمايان رهيون", label=1),
    InputExample(guid=7, text_a="ڪرسٽيانو رونالڊو 2023ع دوران 38 گول ڪري سڀني کان اڳيان نڪري ويو", label=1),

    #Crime
    InputExample(guid=8, text_a="پر گڏيل قومن جي هڪ بيان ۾ چيو ويو آهي ته موت جي سزا ماڻيندڙن جو اصل انگ ان کان وڌيڪ آهي.", label=2),
    InputExample(guid=9, text_a="هڪ عورت کي سندس ڌيءَ سميت ڏوهاري اغوا ڪري ويا.", label=2),
    InputExample(guid=10, text_a="پرڏيهي ميڊيا موجب ان ڳالهه جو انڪشاف هڪ ايراني نائب وزير يونس پناهي ڪيو آهي، جنهن چيو آهي ته ايران ۾ اسڪول جي ڇوڪرين کي زهر ڏنو ويو", label=2),
    InputExample(guid=11, text_a="ڳڙهي خدابخش کوسو ۾ نينگري جي قتل جو الزام.", label=2),
]


In [5]:
# ==============================
# Define Classes
# ==============================
classes = ["Politics", "Sports", "Crime"]
label_map = {"Politics": 0, "Sports": 1, "Crime":2}

# # Step 1: Use load_plm with 'roberta' to get the correct WrapperClass
_, _, _, WrapperClass = load_plm("roberta", "roberta-base")  # Just to get the wrapper

# # Step 2: Manually load XLM-RoBERTa model/tokenizer
model_name = "xlm-roberta-large"
tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
plm = XLMRobertaForMaskedLM.from_pretrained(model_name)

# ==============================
# Load Pretrained Language Model (mBERT)
# ==============================
# plm, tokenizer, model_config, WrapperClass = load_plm("bert", "bert-base-multilingual-cased")


C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 98f030bb-7054-457d-884d-ac51d686432e)')' thrown while requesting HEAD https://huggingface.co/roberta-base/resolve/main/config.json
Retrying in 1s [Retry 1/5].


In [6]:

# ==============================
# Define Prompt Template (Manual)
# ==============================
template = ManualTemplate(
    text = '{"placeholder":"text_a"} هي جملا آهي {"mask"}.', #sindhi
    tokenizer=tokenizer,
)


# automatic verbalizer
verbalizer = AutomaticVerbalizer(
    tokenizer=tokenizer,
    classes=classes,
    num_candidates=1000,
    label_word_num_per_class=5,
)

In [7]:



# ==============================
# Create Prompt Model
# ==============================
prompt_model = PromptForClassification(
    template=template,
    plm=plm,
    verbalizer=verbalizer
)

# ==============================
# DataLoader for Training
# ==============================
train_loader = PromptDataLoader(
    dataset=train_dataset,
    tokenizer=tokenizer,
    template=template,
    tokenizer_wrapper_class=WrapperClass,
    max_seq_length=128,
    batch_size=4,
    shuffle=True  # Even though shuffled, reproducibility is preserved by seeding
)

# ==============================
# Fine-Tuning the Prompt Model
# ==============================
prompt_model.train()
optimizer = AdamW(prompt_model.parameters(), lr=1e-5)

for epoch in range(5):
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        logits = prompt_model(batch)
        loss = torch.nn.CrossEntropyLoss()(logits, batch['label'])
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} Loss: {total_loss:.4f}")

# ==============================
# Load Evaluation Dataset
# ==============================
df = pd.read_csv(r"C:\Users\stdFurqan\Desktop\Sindhi (politics)\split_30.csv")
eval_dataset = [
    InputExample(guid=i, text_a=row['News Headlines'], label=label_map[row['Aspect Category']])
    for i, row in df.iterrows()
]


eval_loader = PromptDataLoader(
    dataset=eval_dataset,
    tokenizer=tokenizer,
    template=template,
    tokenizer_wrapper_class=WrapperClass,
    max_seq_length=128,
    batch_size=8,
    shuffle=False
)

# ==============================
# Evaluate Model
# ==============================
prompt_model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in eval_loader:
        logits = prompt_model(batch)
        preds = torch.argmax(logits, dim=-1)
        all_preds.extend(preds.cpu().tolist())
        all_labels.extend(batch['label'].cpu().tolist())

# ==============================
# Print Classification Report
# ==============================
print("\n📊 Fine-tuned Classification Report:")
print(classification_report(all_labels, all_preds, target_names=classes, digits=4))


tokenizing: 12it [00:00, 1997.21it/s]


Epoch 1 Loss: 5.3057
Epoch 2 Loss: 3.4287
Epoch 3 Loss: 3.8526
Epoch 4 Loss: 3.6253
Epoch 5 Loss: 3.7148


tokenizing: 4967it [00:01, 3867.86it/s]



📊 Fine-tuned Classification Report:
              precision    recall  f1-score   support

    Politics     0.4531    0.3260    0.3792      2282
      Sports     0.2412    0.3149    0.2732      1245
       Crime     0.2935    0.3465    0.3178      1440

    accuracy                         0.3292      4967
   macro avg     0.3293    0.3291    0.3234      4967
weighted avg     0.3537    0.3292    0.3348      4967

